In [10]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)
import math

from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.fileName=pathParts[4]
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        #print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0].tolist()
        self.y=data[:, 1].tolist()
        self.t=data[:, 2].tolist()
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            self.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure) > 0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (len(self.pressure) > 0 and self.pressure[len(self.pressure)-1] == 0):
                self.x.pop(len(self.pressure)-1)
                self.y.pop(len(self.pressure)-1)
                self.t.pop(len(self.pressure)-1)
                self.pressure.pop(len(self.pressure)-1)
        
        self.dx=difference(self.x)
        self.dy=difference(self.y)
        
        self.calculatePathTangentAngle()
        self.calculatePVM()
        self.dpvm=difference(self.pvm)
        self.dpta=difference(self.pta)
        self.lcr=self.calculateLCR()
        self.dlcr=difference(self.lcr)
        
        self.calculateTotalAccelerationMagnitude()
        self.dtam=difference(self.tam)
        #self.pta=math.atan(self.dy/self.dx)
        #self.pvm=math.sqrt(self.dx**2+self.dy*2)
        #self.dx=self.x.diff()
        #self.lcr=math.log(self.pvm/self.pta,10)

        self.x=normalize(self.x)
        self.y=normalize(self.y)
    
        if self.inputDevice != "finger":
            self.dpressure=difference(self.pressure)
            self.dpressure=normalize(self.dpressure)    
            self.pressure=normalize(self.pressure)
        
        self.pvm=normalize(self.pvm)
        self.pta=normalize(self.pta)
        self.tam=normalize(self.tam)
        self.dpvm=normalize(self.dpvm)
        self.dpta=normalize(self.dpta)
        self.dlcr=normalize(self.dlcr)
        
        self.ddx=difference(self.dx)
        self.ddy=difference(self.dy)
        
        '''
        print('self.x') 
        print(self.x) 
        print(self.filePath) 
        print(self.dx)  
        print(self.dy)
        print(self.dpressure)
        print(self.pta)
        print(self.pvm)
        #print(self.lcr)
        print(self.tam)
        '''
    
    def calculatePathTangentAngle(self):
        self.pta=[]
        for i in range(0,len(self.dx)):
            if(self.dx[i]==0):
                self.pta.append(math.atan(self.dy[i]/(self.dx[i] + 0.0000000000000001)))
            else:
                self.pta.append(math.atan(self.dy[i]/self.dx[i]))
        
    def calculatePVM(self):
        self.pvm=[]
        for i in range(0,len(self.dx)):
            self.pvm.append(math.sqrt(self.dx[i]**2+self.dy[i]**2))
            
    def calculateLCR(self):
        lcr=[]
        for i in range(0,len(self.dpta)):
            value=0
            if(self.dpta[i]==0):
                value=self.pvm[i]/(self.dpta[i]+0.0000000000000001)
            else:
                value=self.pvm[i]/self.dpta[i]
            value=abs(value)
            if(value<=0):
                '''
                print("value<=0 at index:")
                print(i)
                print('value:')
                print(value)
                print('self.dx')
                print(self.dx)
                print('self.dy') 
                print(self.dy)
                print('self.pta') 
                print(self.pta)
                print('self.dpta') 
                print(self.dpta)
                print('self.pvm')
                print(self.pvm)
                '''
                lcr.append(math.log(value+0.0000000000000001,10))
            else:
                lcr.append(math.log(value,10))
                
        return lcr
    
    def calculateTotalAccelerationMagnitude(self):
        self.tam=[]
        for i in range(0,len(self.dpvm)):
            self.pta.append(math.sqrt(self.dpvm[i]**2+(self.pvm[i]**2)*(self.pta[i]**2)))
            
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        '''
        print(split)
        print(inputDevice)
        print(signerId)
        '''
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

class Comparison:
    def __init__(self, file1, file2, label, filePath):
        self.file1=file1
        self.file2=file2
        self.label=label
        self.filePath=filePath   
        
def readFilesInDir(dirPath, allFiles):
    print('asdasd')
    signatureFiles = os.listdir(dirPath)
    i=0
    for file in signatureFiles:
        print(i)
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)
        if i%100==0:
            print(i)
        i+=1
        
def readComparisonFile(filePath, comparisons):
    dataframe=pd.read_csv(filePath,sep=' ',header=None)
    array=dataframe.to_numpy()
    #print(array)
    #print(len(array[:, 0]))
    for i in range(0,len(array[:, 0])-1):
        comparison=Comparison(array[i,0],array[i,1],array[i,2],filePath)
        comparisons.append(comparison)
        
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
    return diff

def Average(lst):
    return sum(lst) / len(lst)

def normalizeX(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.x
    
    #print('xAll')
    #print(xAll)
    #average=mean(xAll)
    z = stats.zscore(all)
    #print(average)
    print(z)
    allIndex=0
    
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized x:')
    for item in allFiles:
        print(item.x)
    '''
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=(item.x[i]-average)/z
    '''
def normalizeY(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.y
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.y),1):
            item.y[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized y:')
    for item in allFiles:
        print(item.y)
            
def normalizePressure(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.pressure
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized pressure:')
    for item in allFiles:
        print(item.filePath)
        print(item.pressure)

def normalize(data):
    normalized=[]
    z = stats.zscore(data)
    #print(z)

    for i in range(0,len(data),1):
        normalized.append(z[i])
            
    return normalized
'''
for item in normalized:
        print(item)
        
    return normalized
'''
    
        
def normalizeY2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.y)
        print(z)

        for i in range(0,len(item.y),1):
            item.y[i]=z[i]

    print('Normalized y:')
    for item in allFiles:
        print(item.y)
        
def normalizePressure2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.pressure)
        print('item.pressure:')
        print(item.pressure)
        print('z:')
        print(z)

        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[i]

    print('Normalized pressure:')
    for item in allFiles:
        print("item.filePath")
        print(item.pressure)

def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

def dtwAllX(comparisons,allFiles):
    comparisonCount=len(comparisons)
    for i in range(0,comparisonCount):
        print('findok')
        signature1=findByPath(allFiles,comparisons[i].file1)
        signature2=findByPath(allFiles,comparisons[i].file2)
        print('findok')
        #signature1=next((x for x in allFiles if x.fileName == comparisons[i].file1), None)
        #signature2=next((x for x in allFiles if x.fileName == comparisons[i].file2), None)
        if signature1 is None or signature2 is None:
            print('valamelyik signature nem található')
            print('i:')
            print(comparisons[i].file1)
            print(comparisons[i].file2)
            for file in allFiles:
                print(file.fileName)
        print('dtw')
        #dtwOutputs.append(fastdtw(signature1.x, signature2.x, dist=euclidean))
        distance, comparisons[i].dtwx=fastdtw(signature1.x, signature2.x, dist=euclidean)
        distance, comparisons[i].dtwy=fastdtw(signature1.y, signature2.y, dist=euclidean)
        if(signature1.inputDevice=='stylus'):
            distance, comparisons[i].dtwpressure=fastdtw(signature1.pressure, signature2.pressure, dist=euclidean)
            distance, comparisons[i].dtwdpressure=fastdtw(signature1.dpressure, signature2.dpressure, dist=euclidean)
        distance, comparisons[i].dtwdx=fastdtw(signature1.dx, signature2.dx, dist=euclidean)
        distance, comparisons[i].dtwdy=fastdtw(signature1.dy, signature2.dy, dist=euclidean)
        distance, comparisons[i].dtwpta=fastdtw(signature1.pta, signature2.pta, dist=euclidean)
        distance, comparisons[i].dtwpvm=fastdtw(signature1.pvm, signature2.pvm, dist=euclidean)
        
        distance, comparisons[i].dtwdpvm=fastdtw(signature1.dpvm, signature2.dpvm, dist=euclidean)
        distance, comparisons[i].dtwdpta=fastdtw(signature1.dpta, signature2.dpta, dist=euclidean)
        distance, comparisons[i].dtwlcr=fastdtw(signature1.lcr, signature2.lcr, dist=euclidean)
        distance, comparisons[i].dtwdlcr=fastdtw(signature1.dlcr, signature2.dlcr, dist=euclidean)
        distance, comparisons[i].dtwtam=fastdtw(signature1.tam, signature2.tam, dist=euclidean)
        distance, comparisons[i].dtwdtam=fastdtw(signature1.dtam, signature2.dtam, dist=euclidean)
        
        distance, comparisons[i].dtwddx=fastdtw(signature1.ddx, signature2.ddx, dist=euclidean)
        distance, comparisons[i].dtwddy=fastdtw(signature1.ddy, signature2.ddy, dist=euclidean) 
        #comparisons[i].dtwx=np.array(comparisons[i].dtwx,np.dtype('int,int'))
        normalizer = tf.keras.layers.Normalization(axis=None)
        comparisons[i].dtwx=convertTuplesToMatrix(comparisons[i].dtwx)
        comparisons[i].dtwx=comparisons[i].dtwx.astype("float32")
        #comparisons[i].dtwx=normalizer.adapt(comparisons[i].dtwx)
        #comparisons[i].dtwx = stats.zscore(comparisons[i].dtwx)
        #print('dtw')
        #print(comparisons[i].dtwddx)
        print(str(i)+'/'+str(comparisonCount))
        if i==10:
            return

def findByPath(allFiles,filePath):
    for file in allFiles:
        if file.fileName==filePath:
            return file
        
def findByPath(allFiles,filePath):
    for file in allFiles:
        if file.fileName==filePath:
            return file
         
def convertTuplesToMatrix(tupleList):
    array=np.array(int)
    for t in tupleList:
        array=np.append(array, t[0])
        array=np.append(array, t[1])
    
    array=np.delete(array,0)
    max=np.amax(array)
    
    for i in range(array.size):
        array[i]=array[i]/max
        
    return array         

allFiles=[]
   
print('Reading signature files started')
#readFilesInDir('C:/DeepSignDB/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmallc/Evaluation/finger',allFiles)
#readFilesInDir('C:/DeepSignDB/Development/stylus',allFiles)
#readFilesInDir('C:/DeepSignDB/Development/finger',allFiles)
print('Reading signature files done')

comparisonFiles=[]
print('Reading comparison files started')
#readComparisonFile('C:/DeepSignDB/Comparison_Files/stylus/1vs1/random/Comp_DeepSignDB_random_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('C:/DeepSignDB/Comparison_Files/stylus/1vs1/skilled/Comp_DeepSignDB_skilled_stylus_1vs1.txt',comparisonFiles)
readComparisonFile('D:/DeepSignSmallc/Comparison_Files/finger/1vs1/random/Comp_DeepSignDB_random_finger_1vs1.txt',comparisonFiles)
#readComparisonFile('C:/DeepSignDB/Comparison_Files_small/finger/1vs1/skilled/Comp_DeepSignDB_skilled_finger_1vs1.txt',comparisonFiles)
print('Reading comparison files done')

'''
readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)

#readComparisonFile('D:/DeepSignSmall/Comparison_Files/stylus/1vs1/random/Comp_DeepSignDB_random_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('D:/DeepSignSmall/Comparison_Files/stylus/1vs1/skilled/Comp_DeepSignDB_skilled_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('D:/DeepSignSmall/Comparison_Files/finger/1vs1/random/Comp_DeepSignDB_random_finger_1vs1.txt',comparisonFiles)
readComparisonFile('D:/DeepSignSmall/Comparison_Files/finger/1vs1/skilled/Comp_DeepSignDB_skilled_finger_1vs1.txt',comparisonFiles)
'''
'''
print('Comparison files:')
for comparisonFile in comparisonFiles:
    print(comparisonFile.file1)
    print(comparisonFile.file2)
    print(comparisonFile.label)
    print(comparisonFile.filePath)
'''

'''
for file in allFiles:
    #print(file.data)
    #print(file.filePath)
    #print(file.db)
    #print(file.pressure)
'''

print('Allfiles:')
#print(allFiles)
print('Loading data finished')
dtwMatrix=dtwAllX(comparisonFiles,allFiles)
print(dtwMatrix)

train_signatures=[]
train_labels=[]
test_signatures=[]
test_labels=[]
#train_labels=np.empty(0, dtype=float)
train_labels=[]
for comparisonFile in comparisonFiles:
    #ez volt train_signatures.append(tf.convert_to_tensor(comparisonFile.dtwx.reshape(1,-1)))
    #train_labels=np.append(train_labels,np.array(comparisonFile.label))
    #train_labels.append(np.array(comparisonFile.label))
    train_labels.append(comparisonFile.label)
    print('train_labels')
    print(train_labels)
    #test_signatures.append(comparisonFile.file1)
    #test_labels.append(comparisonFile.label)



#train_labels=np.array(train_labels)
#train_signatures=np.array(train_signatures)
train_labels=np.asarray(train_labels, dtype="uint8")


train_labels=tf.data.Dataset.from_tensor_slices(train_labels)
train_signatures=tf.data.Dataset.from_tensors(train_signatures)

#train_signatures=tf.convert_to_tensor(train_signatures)
#train_labels=train_labels.reshape(1,-1)
#train_labels=tf.convert_to_tensor(train_labels)

print('train_signatures')
print(train_signatures)
print('train_labels')
print(train_labels)

#train_signatures=np.asarray(train_signatures).astype(np.float32)
#train_labels=np.asarray(train_labels).astype(np.float32)

#normalizeX2(allFiles)
#normalizeY2(allFiles)
#normalizePressure2(allFiles)

model = tf.keras.models.Sequential([
tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True,input_shape=(1, 764))),
tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True)),
tf.keras.layers.Dense(2, activation='sigmoid')
],name="Sequential_model")
'''
embedding=tf.keras.layers.Embedding(input_dim=100,output_dim=64,mask_zero=True);
bgru1=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True), input_shape=(5, 10))
bgru2=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True), input_shape=(5, 10))
#concatenateLayer=tf.keras.layers.concatenate([bgru1,bgru2],axis=1)
layer3=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(2, return_sequences=True), input_shape=(5, 10))
ff=tf.keras.layers.Dense(2, activation='sigmoid')

#model.add(embedding)
model.add(bgru1)
model.add(bgru2)
#model.add(concatenate)
model.add(ff)
'''

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
'''
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),0
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
'''

#predictions = model(train_signatures[:1]).numpy()
#predictions

#tf.nn.softmax(predictions).numpy()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#loss_fn(train_labels[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

label=np.empty(0, dtype=float)
label=np.append(label,np.float32(comparisonFiles[0].label))
model.fit(train_signatures, train_labels, epochs=5)

model.evaluate(train_signatures, train_labels, verbose=2)

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(train_signatures[:5])


3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
TensorFlow version: 2.8.0
Reading signature files started
asdasd
0
0
1
2
3
4
Reading signature files done
Reading comparison files started
Reading comparison files done
Allfiles:
Loading data finished
findok
findok
dtw
0/3
findok
findok
dtw
1/3
findok
findok
dtw
2/3
None
train_labels
[0]
train_labels
[0, 0]
train_labels
[0, 0, 1]
train_signatures
<TensorDataset element_spec=TensorSpec(shape=(0,), dtype=tf.float32, name=None)>
train_labels
<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.uint8, name=None)>


ValueError: `y` argument is not supported when using dataset as input.